In [12]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path
from datetime import datetime



In [13]:
import os

print("PROJECT_CONNECTION_STRING:", os.environ.get("PROJECT_CONNECTION_STRING"))
print("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME:", os.environ.get("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"))

PROJECT_CONNECTION_STRING: brazilsouth.api.azureml.ms;abd5f748-2340-4f7c-83de-18ffddb67814;rg-jefersonguevara245999-8052_ai;paradoja
AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME: None


In [14]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ["PROJECT_CONNECTION_STRING"]
    )
    print("Conexión al proyecto exitosa.")
except Exception as e:
    print(f"Error en la conexión al proyecto: {e}")

Conexión al proyecto exitosa.


In [15]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

In [ ]:
from IPython.display import display, HTML, Image
from pathlib import Path


async def run_agent_with_visualization():
    html_output = "<h2>Azure AI Agent Execution</h2>"

    with project_client:
        # Crear una instancia de la CodeInterpreterTool
        code_interpreter = CodeInterpreterTool()

        # The CodeInterpreterTool Necesita ser incluido en la creación del agente.
        # Asegúrese de configurar el nombre del modelo correcto tal como se implementó en Azure AI Foundry para su caso de uso
        agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="gpt-4o-mini",
            instructions="Eres un agente útil.",
            tools=code_interpreter.definitions,
            tool_resources=code_interpreter.resources,
        )
        html_output += f"<div><strong>Agente creado</strong> with ID: {agent.id}</div>"

        # Create a thread
        thread = project_client.agents.create_thread()
        html_output += f"<div><strong>Hilo creado</strong> with ID: {thread.id}</div>"

        # User query - display nicely
        user_query = "¿Podría crear un gráfico de barras para el beneficio operativo con los siguientes datos y proporcionarme el archivo? Bali: 100 viajeros, París: 356 viajeros, Londres: 900 viajeros, Tokio: 850 viajeros"
        html_output += "<div style='margin:15px 0; padding:10px; background-color:#f5f5f5; border-left:4px solid #007bff; border-radius:4px;'>"
        html_output += "<strong>User:</strong><br>"
        html_output += f"<div style='margin-left:15px'>{user_query}</div>"
        html_output += "</div>"

        # crear un mensaje
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_query,
        )

        # Ejecutar el agente: mostrar un mensaje de "procesando"
        display(HTML(
            html_output + "<div style='color:#007bff'><i>Solicitud de procesamiento...</i></div>"))

        # Execute the run
        run = project_client.agents.create_and_process_run(
            thread_id=thread.id, agent_id=agent.id)

        # Update status
        status_color = 'green' if run.status == 'completed' else 'red'
        html_output += f"<div><strong>Arranque terminado</strong> with status: <span style='color:{status_color}'>{run.status}</span></div>"

        if run.status == "failed":
            html_output += f"<div style='color:red'><strong>Arranque fallido:</strong> {run.last_error}</div>"

        # Recibir mensajes del hilo
        messages = project_client.agents.list_messages(thread_id=thread.id)

        # Dar formato a la respuesta del asistente
        html_output += "<div style='margin:15px 0; padding:10px; background-color:#f0f7ff; border-left:4px solid #28a745; border-radius:4px;'>"
        html_output += "<strong>Assistant:</strong><br>"

        # Manejar mensajes según la estructura real
        # Primero, intenta obtener las respuestas de texto del asistente.
        try:
            # Primer enfoque: si los mensajes son una lista de objetos con atributo de rol
            assistant_msgs = [msg for msg in messages if hasattr(
                msg, 'role') and msg.role == "assistant"]

            if assistant_msgs:
                last_msg = assistant_msgs[-1]
                if hasattr(last_msg, 'content'):
                    if isinstance(last_msg.content, list):
                        for content_item in last_msg.content:
                            if hasattr(content_item, 'type') and content_item.type == "text":
                                html_output += f"<div style='margin-left:15px; white-space:pre-wrap'>{content_item.text.value}</div>"
                    elif isinstance(last_msg.content, str):
                        html_output += f"<div style='margin-left:15px; white-space:pre-wrap'>{last_msg.content}</div>"

            # Si no se encontraron mensajes con el enfoque anterior, pruebe una estructura diferente
            if not assistant_msgs:
                # Si los mensajes son una clase con atributos
                if hasattr(messages, 'data'):
                    for msg in messages.data:
                        if hasattr(msg, 'role') and msg.role == "assistant":
                            if hasattr(msg, 'content'):
                                html_output += f"<div style='margin-left:15px; white-space:pre-wrap'>{msg.content}</div>"

        except Exception as e:
            html_output += f"<div style='color:red'><strong>Error processing messages:</strong> {str(e)}</div>"

        html_output += "</div>"

        # Manejar el contenido de la imagen según la estructura real
        saved_images = []
        try:
            # Try to access image_contents as an attribute
            if hasattr(messages, 'image_contents'):
                for image_content in messages.image_contents:
                    file_id = image_content.image_file.file_id
                    file_name = f"{file_id}_image_file.png"
                    project_client.agents.save_file(
                        file_id=file_id, file_name=file_name)
                    saved_images.append(file_name)
                    html_output += f"<div style='margin-top:10px'><strong>Generated Image:</strong> {file_name}</div>"
        except Exception as e:
            html_output += f"<div style='color:orange'><i>Note: No images found or error processing images</i></div>"

        # Handle file path annotations based on the actual structure
        try:
            # Try to access file_path_annotations as an attribute
            if hasattr(messages, 'file_path_annotations'):
                for file_path_annotation in messages.file_path_annotations:
                    file_name = Path(file_path_annotation.text).name
                    project_client.agents.save_file(
                        file_id=file_path_annotation.file_path.file_id, file_name=file_name)
                    html_output += "<div style='margin:10px 0; padding:8px; background-color:#f8f9fa; border:1px solid #ddd; border-radius:4px;'>"
                    html_output += f"<strong>Generated File:</strong> {file_name}<br>"
                    html_output += f"<strong>Type:</strong> {file_path_annotation.type}<br>"
                    html_output += "</div>"
        except Exception as e:
            html_output += f"<div style='color:orange'><i>Note: No file annotations found or error processing files</i></div>"

        # Delete the agent once done
        project_client.agents.delete_agent(agent.id)
        html_output += "<div style='margin-top:10px'><i>Agent deleted after completion</i></div>"

        # Final display of all content
        display(HTML(html_output))

        # Display any saved images
        for img_file in saved_images:
            display(Image(img_file))

# Execute the function
await run_agent_with_visualization()

HttpResponseError: () Agents is not enabled in current region.
Code: 
Message: Agents is not enabled in current region.